# WIP: Create some more analysis here

In [40]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [41]:
plt.rcParams["figure.figsize"] = (14, 7)

In [2]:
df_affiliation = pd.read_csv("affiliation.csv")

In [4]:
df_nominating_organization = pd.read_csv("nominating_organization.csv")

In [6]:
df_election = pd.read_csv("election.csv")

In [8]:
df_district_hierarchy = pd.read_csv("district_hierarchy.csv")

In [10]:
df_geographical_unit = pd.read_csv("geographical_unit.csv")

In [12]:
df_election_precinct = pd.read_csv("election_precinct.csv")

In [14]:
df_vote_record = pd.read_csv("vote_record.csv")

In [15]:
df_vote_record.head(3).T

,0,1,2
vid,0,1,2
vote_count,19,45,163
organization,MSZDP,MSZP,SZDSZ
precinct_id,0,0,0


In [19]:
party_level_df = df_vote_record.merge(df_affiliation).merge(df_election_precinct)

In [35]:
# only first rounds, non individual
relevant_elections = df_election.loc[
    lambda df: ~df["is_individual"] & df["election_id"].str.endswith("1")
]

In [43]:
# parties that have received over 10% of the votes in 3 different election
parties_for_elections = (
    party_level_df.pivot_table(
        index="party", columns="election_id", values="vote_count", aggfunc="sum"
    )
    .loc[:, relevant_elections["election_id"]]
    .fillna(0)
)

In [53]:
total_votes_by_election = df_vote_record.merge(df_election_precinct).groupby("election_id")["vote_count"].sum()

In [70]:
(
    parties_for_elections.loc[
        lambda df: (df / total_votes_by_election[df.columns] > 0.03).sum(axis=1) > 2
    ]
    .rename(columns=relevant_elections.set_index("election_id")["start_date"].to_dict())
    .astype(int)
    .T.style.background_gradient(
        axis=None,
    )
)

party,FIDESZ,FKGP,JOBBIK,KDNP,LMP,MDF,MSZP,MUNKÁSPÁRT,SZDSZ
election_id,,,,,,,,,
1990-03-25,439481,576256,0,317183,0,1213825,534898,180899,1050452
1994-05-08,379295,476416,0,379573,0,633957,1781867,172117,1066074
1998-05-10,1263522,617740,0,116065,0,139934,1445909,183064,353186
2002-04-07,2306763,42338,0,0,0,2306763,2361997,121503,313084
2006-04-09,2272979,838,119007,2272979,0,272831,2336705,21955,351612
2010-04-11,2706292,0,855436,2706292,383876,136895,990428,5606,0
2014-04-06,2142142,8040,1017550,2142142,268840,0,1289311,28260,0
2018-04-08,2607990,0,1090751,2607990,402347,0,681454,15595,0


In [69]:
(
    parties_for_elections.pipe(
        lambda df: (df / total_votes_by_election[df.columns])
    ).loc[lambda df: (df > 0.03).sum(axis=1) > 2]
    .rename(columns=relevant_elections.set_index("election_id")["start_date"].to_dict())
    .T.style.background_gradient(
        axis=None,
    )
)

party,FIDESZ,FKGP,JOBBIK,KDNP,LMP,MDF,MSZP,MUNKÁSPÁRT,SZDSZ
election_id,,,,,,,,,
1990-03-25,0.089512,0.117369,0.000000,0.064602,0.000000,0.247226,0.108946,0.036845,0.213951
1994-05-08,0.070227,0.088210,0.000000,0.070279,0.000000,0.117379,0.329917,0.031868,0.197386
1998-05-10,0.281786,0.137766,0.000000,0.025884,0.000000,0.031208,0.322462,0.040826,0.078766
2002-04-07,0.410694,0.007538,0.000000,0.000000,0.000000,0.410694,0.420527,0.021632,0.055741
2006-04-09,0.420295,0.000155,0.022006,0.420295,0.000000,0.050449,0.432079,0.004060,0.065016
2010-04-11,0.527282,0.000000,0.166669,0.527282,0.074793,0.026672,0.192971,0.001092,0.000000
2014-04-06,0.437226,0.001641,0.207689,0.437226,0.054872,0.000000,0.263157,0.005768,0.000000
2018-04-08,0.476805,0.000000,0.199416,0.476805,0.073559,0.000000,0.124586,0.002851,0.000000
